In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1603645659758_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [17]:
from pyspark.sql.functions import col

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://fall2020-cse6242"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp,lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
def long_trips(trips):
    # Returns a Dataframe with Schema the same as :trips:
   trips = trips.filter(trips.trip_distance>=2.0) 
   return trips

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
def manhattan_trips(trips, lookup):
    # Returns a Dataframe with Schema: DOLocationID, pcount
    data = trips.join(lookup,trips.DOLocationID == lookup.LocationID, "left").select(trips.DOLocationID, trips.passenger_count,lookup.Borough, lookup.Zone, lookup.service_zone)
    data1 = data.filter(data.Borough == "Manhattan").groupby(data.DOLocationID).agg({"Passenger_count":"sum"}).orderBy(col("sum(Passenger_count)").desc()).withColumnRenamed("sum(Passenger_count)","pcount").limit(20)  
    return data1 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
def weighted_profit(trips, mtrips): 
    # Returns a Dataframe with Schema: PULocationID, weighted_profit
    data1 = trips.groupby("PULocationID").agg({'total_amount':"avg" }) 
    data2 = trips.groupby("PULocationID").count().withColumnRenamed("count","total_count").withColumnRenamed("PULocationID","PUL")
    data3 = trips.filter(col("DOLocationID").isin(mtrips.DOLocationID)).groupby("PULocationID").count()
    
    data4 = data2.join(data3, data2.PUL == data3.PULocationID, "left").withColumn("prop", col("count")/col("total_count")).select("PUL","prop")
    
    data5 = data4.join(data1, data4.PUL == data1.PULocationID, "left").withColumn("weighted", col("prop")*col("avg(total_amount)")).select("PUL","weighted").withColumnRenamed("PUL", "PULocationID").withColumnRenamed("weighted","weighted_profit")
    return data5

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
def final_output(calc, lookup): 
    # Returns a Dataframe with Schema: Zone, Borough, weighted_profit
    # Returns a Dataframe with Schema: Zone, Borough, weighted_profit
    data = calc.join(lookup,calc.PULocationID == lookup.LocationID, "left").orderBy(col("weighted_profit").desc()).select("Zone","Borough","weighted_profit").limit(20)
    return data

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…